In [1]:
from __future__ import print_function, division

import numpy as np
import sys
import os
import json
import pandas as pd


# To make rows in tables sorted in order 
def custom_sort(x):
    if 'no_clean' in x:
        return(1, x)
    all_cl_methods = ['argmax', 'conf_joint_only', 'cl_pbc', 'cl_pbnr', 'cl_both', 'cl']
    if x in all_cl_methods:
        return (2, str(all_cl_methods.index(x))+x)
    return (3, x)

In [2]:
df = pd.read_csv('mask_results.csv')
df = df.rename({'noise_amount':'noise', 'frac_zero_noise_rates':'sparsity'}, axis=1)

cols_keep = ['method', 'noise', 'sparsity']
metrics = ['accuracy', 'f1']
dfs_acc, dfs_f1 = {}, {}
dfs = [dfs_acc, dfs_f1]
row_order = list(np.unique(df['method']))
row_order.sort(key=custom_sort)


for i in range(len(metrics)):
    dfs_metric = dfs[i]
    metric = metrics[i]
    
    for seed in np.unique(df['seed']):
        df_metric_seed = df[df['seed']==seed][[metric]+cols_keep]
        df_metric_seed = pd.concat([
            z.sort_values(by=['noise', 'sparsity']).set_index(
                ['noise', 'sparsity']).drop(
                ['method'], axis=1).T.set_index([[i]]) \
            for i, z in df_metric_seed.groupby(['method'])
        ])
        df_metric_seed = df_metric_seed.reindex(row_order)
        df_metric_seed = df_metric_seed.rename({'no_clean': 'no clean', 'cl':'CL (best)', 'kmeans':'CTRL'}, axis=0)
        dfs_metric[seed] = df_metric_seed * 100

In [3]:
dfs_score = dfs_acc
dfs_score_mean = (pd.concat(dfs_score.values())).mean(level=0)
dfs_score_std = (pd.concat(dfs_score.values())).std(level=0)
table = dfs_score_mean.round(1).astype(str) + '±' + dfs_score_std.round(1).astype(str)
table

noise            0.0       0.1                                     0.2  \
sparsity         0.0       0.0       0.2       0.4       0.6       0.0   
no clean   100.0±0.0  90.1±0.0  90.1±0.0  90.1±0.0  90.0±0.0  80.1±0.0   
CL (best)   98.0±0.0  93.2±0.0  93.4±0.2  93.6±0.3  93.3±0.5  92.6±0.1   
CTRL        97.9±0.1  98.6±0.1  98.4±0.3  98.6±0.3  98.4±0.2  97.4±0.2   

noise                                    
sparsity        0.2       0.4       0.6  
no clean   80.1±0.0  80.1±0.0  80.0±0.0  
CL (best)  92.9±0.6  92.6±0.2  92.8±0.4  
CTRL       97.5±0.1  97.1±0.3  96.2±1.6

In [4]:
dfs_score = dfs_f1
dfs_score_mean = (pd.concat(dfs_score.values())).mean(level=0)
dfs_score_std = (pd.concat(dfs_score.values())).std(level=0)
table = dfs_score_mean.round(1).astype(str) + '±' + dfs_score_std.round(1).astype(str)
table

noise          0.0       0.1                                     0.2  \
sparsity       0.0       0.0       0.2       0.4       0.6       0.0   
no clean   0.0±0.0   0.0±0.0   0.0±0.0   0.0±0.0   0.0±0.0   0.0±0.0   
CL (best)  0.0±0.0  73.0±0.1  73.2±0.6  73.4±1.0  72.1±1.6  83.1±0.3   
CTRL       0.0±0.0  92.3±0.4  91.2±1.7  92.5±1.7  91.9±0.9  93.4±0.8   

noise                                    
sparsity        0.2       0.4       0.6  
no clean    0.0±0.0   0.0±0.0   0.0±0.0  
CL (best)  83.8±1.2  83.1±0.6  83.6±1.0  
CTRL       93.6±0.3  93.0±0.5  89.9±5.1

In [5]:
test_ckpt = 'last'

df = pd.read_csv('evaluation.csv')
df = df.rename({'noise_amount':'noise', 'frac_zero_noise_rates':'sparsity'}, axis=1)

cols_keep = ['method', 'noise', 'sparsity', 'acc1']
dfs = {}
row_order = list(np.unique(df['method']))
row_order.sort(key=custom_sort)


for seed in np.unique(df['seed']):
    df_seed = df[(df['seed']==seed) & (df['test_ckpt']==test_ckpt)][cols_keep]
    df_seed = pd.concat([
        z.sort_values(by=['noise', 'sparsity']).set_index(
            ['noise', 'sparsity']).drop(
            ['method'], axis=1).T.set_index([[i]]) \
        for i, z in df_seed.groupby(['method'])
    ])
    df_seed = df_seed.reindex(row_order)
    dfs[seed] = df_seed

dfs_score = dfs
dfs_score_mean = (pd.concat(dfs_score.values())).mean(level=0)
dfs_score_std = (pd.concat(dfs_score.values())).std(level=0)
table = dfs_score_mean.round(1).astype(str) + '±' + dfs_score_std.round(1).astype(str)
table

noise          0.0       0.1                                     0.2  \
sparsity       0.0       0.0       0.2       0.4       0.6       0.0   
no_clean  95.4±0.1  89.5±0.6  89.1±0.4  88.9±0.4  89.3±0.5  80.0±0.6   
cl        95.2±0.3  93.6±0.2  93.5±0.2  93.5±0.4  93.3±0.1  92.1±0.4   
kmeans    95.1±0.1  94.3±0.1  94.0±0.5  93.9±0.4  94.2±0.2  93.0±0.5   

noise                                   
sparsity       0.2       0.4       0.6  
no_clean  80.0±0.7  79.9±0.4  79.2±1.0  
cl        92.4±0.3  92.2±0.3  92.6±0.5  
kmeans    93.0±0.2  93.4±0.2  92.2±2.2

In [6]:
test_ckpt = 'best'

df = pd.read_csv('evaluation.csv')
df = df.rename({'noise_amount':'noise', 'frac_zero_noise_rates':'sparsity'}, axis=1)

cols_keep = ['method', 'noise', 'sparsity', 'acc1']
dfs = {}
row_order = list(np.unique(df['method']))
row_order.sort(key=custom_sort)


for seed in np.unique(df['seed']):
    df_seed = df[(df['seed']==seed) & (df['test_ckpt']==test_ckpt)][cols_keep]
    df_seed = pd.concat([
        z.sort_values(by=['noise', 'sparsity']).set_index(
            ['noise', 'sparsity']).drop(
            ['method'], axis=1).T.set_index([[i]]) \
        for i, z in df_seed.groupby(['method'])
    ])
    df_seed = df_seed.reindex(row_order)
    dfs[seed] = df_seed

dfs_score = dfs
dfs_score_mean = (pd.concat(dfs_score.values())).mean(level=0)
dfs_score_std = (pd.concat(dfs_score.values())).std(level=0)
table = dfs_score_mean.round(1).astype(str) + '±' + dfs_score_std.round(1).astype(str)
table

noise          0.0       0.1                                     0.2  \
sparsity       0.0       0.0       0.2       0.4       0.6       0.0   
no_clean  95.5±0.2  90.2±0.4  90.4±0.4  90.1±0.3  90.8±0.4  88.6±0.5   
cl        95.3±0.3  93.7±0.1  93.6±0.2  93.7±0.3  93.4±0.2  92.2±0.3   
kmeans    95.2±0.1  94.4±0.2  94.2±0.5  94.0±0.3  94.3±0.2  93.1±0.4   

noise                                   
sparsity       0.2       0.4       0.6  
no_clean  88.8±0.3  88.8±0.3  89.5±0.3  
cl        92.5±0.3  92.2±0.2  92.7±0.5  
kmeans    93.1±0.2  93.5±0.2  92.8±1.5